In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np

start = '2020-01-01'
end = '2025-03-31'

# ✅ DOWNLOAD EACH TICKER SEPARATELY — DO NOT COMBINE
tsla = yf.Ticker("TSLA").history(start=start, end=end)
sp500 = yf.Ticker("^GSPC").history(start=start, end=end)
t_bill = yf.Ticker("^IRX").history(start=start, end=end)

# ✅ Check for flat columns
print("TSLA Columns:", tsla.columns)





In [ ]:
print("TSLA:", tsla.shape, "from", tsla.index.min(), "to", tsla.index.max())
print("SP500:", sp500.shape, "from", sp500.index.min(), "to", sp500.index.max())
print("T-Bill:", t_bill.shape, "from", t_bill.index.min(), "to", t_bill.index.max())






In [ ]:
# Remove timezone info to make index comparable
tsla.index = tsla.index.tz_localize(None)
sp500.index = sp500.index.tz_localize(None)
t_bill.index = t_bill.index.tz_localize(None)

# Recompute common index
common_idx = tsla.index.intersection(sp500.index).intersection(t_bill.index)

# Filter to common dates
tsla = tsla.loc[common_idx]
sp500 = sp500.loc[common_idx]
t_bill = t_bill.loc[common_idx]


In [ ]:
# Calculate returns
tsla['Returns'] = np.log(tsla['Close'] / tsla['Close'].shift(1))
sp500['Returns'] = np.log(sp500['Close'] / sp500['Close'].shift(1))
t_bill['Daily_RF'] = (t_bill['Close'] / 100) / 252

# Merge final dataset
data = pd.DataFrame({
    'TSLA_Return': tsla['Returns'],
    'SP500_Return': sp500['Returns'],
    'RiskFree': t_bill['Daily_RF']
}).dropna()

# Preview
print(data.tail())
print(data.shape)


**Phase 2**: Exploratory Data Analysis

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(14, 5))
plt.plot(data.index, data['TSLA_Return'], label='TSLA Daily Log Return')
plt.plot(data.index, data['SP500_Return'], label='S&P 500 Daily Log Return', alpha=0.6)
plt.title("Daily Log Returns (TSLA vs S&P 500)")
plt.xlabel("Date")
plt.ylabel("Log Return")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
plt.figure(figsize=(14, 5))
rolling_vol = data['TSLA_Return'].rolling(window=30).std()
plt.plot(data.index, rolling_vol, color='red')
plt.title("Rolling 30-Day Volatility of TSLA")
plt.xlabel("Date")
plt.ylabel("Volatility (Std Dev)")
plt.grid(True)
plt.show()


In [ ]:
plt.figure(figsize=(8, 4))
plt.hist(data['TSLA_Return'], bins=50, color='skyblue', edgecolor='black')
plt.title("Histogram of TSLA Daily Log Returns")
plt.xlabel("Return")
plt.ylabel("Frequency")
plt.grid(True)
plt.show()


In [ ]:
q1_2025 = data['2025-01-01':'2025-03-31']

plt.figure(figsize=(14, 5))
plt.plot(q1_2025.index, q1_2025['TSLA_Return'], label='TSLA Q1 2025')
plt.title("TSLA Daily Returns in Q1 2025")
plt.xlabel("Date")
plt.ylabel("Log Return")
plt.grid(True)
plt.show()


**Phase 3: Implementation**

In [ ]:
from statsmodels.tsa.stattools import adfuller

adf_result = adfuller(data['TSLA_Return'])
print(f"ADF Statistic: {adf_result[0]:.4f}")
print(f"p-value: {adf_result[1]:.4f}")


In [ ]:
from statsmodels.stats.diagnostic import acorr_ljungbox

ljungbox_result = acorr_ljungbox(data['TSLA_Return'], lags=[10], return_df=True)
print(ljungbox_result)


In [ ]:
from statsmodels.tsa.regime_switching.markov_regression import MarkovRegression

# Clean and set frequency
returns_tsla = data["TSLA_Return"].sort_index()
returns_tsla.index = pd.to_datetime(returns_tsla.index)
returns_tsla = returns_tsla.asfreq("B")      # business‑day grid
returns_tsla = returns_tsla.dropna()         # VERY important for stability

# Fit 2‑regime model (no trend)
mod = MarkovRegression(
    returns_tsla,
    k_regimes=2,
    trend="n",
    switching_variance=True,
)

# Use a slightly more robust fitting strategy
res = mod.fit(em_iter=10, search_reps=20, disp=False)
print(res.summary())


In [ ]:
# Means
mean_0 = res.params.iloc[0]
mean_1 = res.params.iloc[1]

# Standard deviations (square root of variance)
std_0 = np.sqrt(res.params.iloc[2])
std_1 = np.sqrt(res.params.iloc[3])

print(f"Regime 0 (Low Volatility): mean = {mean_0:.4f}, std = {std_0:.4f}")
print(f"Regime 1 (High Volatility): mean = {mean_1:.4f}, std = {std_1:.4f}")



In [ ]:
from scipy.special import expit  # logistic function

# Get last 2 parameters from res.params (these are logits)
logit_p00 = res.params.iloc[-2]
logit_p11 = res.params.iloc[-1]

# Convert logits to probabilities
p00 = expit(logit_p00)   # = 1 / (1 + exp(-logit_p00))
p11 = expit(logit_p11)

# Construct transition matrix
transition_matrix = np.array([[p00, 1 - p00],
                              [1 - p11, p11]])

print("Estimated Transition Matrix:")
print(transition_matrix)



In [ ]:
!pip install arch


In [ ]:
from arch import arch_model

# Scale returns by 100 for stability (common in GARCH modeling)
returns_scaled = data['TSLA_Return'] * 100

# Fit GARCH(1,1)
garch_model = arch_model(returns_scaled, vol='GARCH', p=1, q=1)
garch_res = garch_model.fit(disp='off')

# Summary
print(garch_res.summary())


In [ ]:
# Forecasting with GARCH
garch_forecasts = garch_res.forecast(horizon=20)
vol_forecast_1d = np.sqrt(garch_forecasts.variance.values[-1, 0])
vol_forecast_5d = np.sqrt(np.sum(garch_forecasts.variance.values[-1, :5]))
vol_forecast_20d = np.sqrt(np.sum(garch_forecasts.variance.values[-1, :20]))

print("GARCH Forecast Volatility:")
print(f"1-day ahead:  {vol_forecast_1d:.4f}")
print(f"5-day ahead:  {vol_forecast_5d:.4f}")
print(f"20-day ahead: {vol_forecast_20d:.4f}")


In [ ]:
# Approximate forecast using most likely current regime
smoothed_probs = res.smoothed_marginal_probabilities
last_prob_regime0 = smoothed_probs.iloc[-1, 0]
last_prob_regime1 = smoothed_probs.iloc[-1, 1]

# Weighted average volatility
regime_vol_0 = std_0
regime_vol_1 = std_1

vol_regime_switch_1d = last_prob_regime0 * regime_vol_0 + last_prob_regime1 * regime_vol_1

print("Markov Switching Approx Volatility (1-day):", round(vol_regime_switch_1d, 4))


In [ ]:
from scipy.stats import norm

# GARCH VaR (1-day)
var_garch_95 = -vol_forecast_1d * norm.ppf(0.95) / 100  # divide by 100 to undo scaling
print(f"GARCH(1,1) 1-day 95% VaR: {-var_garch_95:.4f}")

# Markov model VaR (1-day)
var_ms_95 = -vol_regime_switch_1d * norm.ppf(0.95)
print(f"Markov Switching 1-day 95% VaR: {-var_ms_95:.4f}")


In [ ]:
# Expected Shortfall (1-day, 95%) for GARCH and MS
es_garch_95 = vol_forecast_1d * norm.pdf(norm.ppf(0.05)) / (0.05 * 100)
es_ms_95 = vol_regime_switch_1d * norm.pdf(norm.ppf(0.05)) / 0.05

print(f"GARCH(1,1) 1-day 95% Expected Shortfall: {es_garch_95:.4f}")
print(f"Markov Switching 1-day 95% Expected Shortfall: {es_ms_95:.4f}")


In [ ]:
# Forecast 1-day-ahead VaR for entire return series
rolling_var = -garch_res.conditional_volatility / 100 * norm.ppf(0.95)

# Compare actual returns vs VaR (violations = actual < VaR)
violations = data['TSLA_Return'].iloc[-len(rolling_var):] < -rolling_var
num_violations = violations.sum()
total_days = len(rolling_var)
violation_rate = num_violations / total_days

print(f"Observed Violations: {num_violations}/{total_days}")
print(f"Violation Rate: {violation_rate:.4f}")


In [ ]:
from scipy.stats import chi2
import numpy as np

# Kupiec's test statistic
alpha = 0.05

# Guard for edge cases where the theoretical likelihood hits log(0)
if num_violations == 0 or num_violations == total_days:
    LR_uc = np.inf
    p_value = 0.0
else:
    expected = alpha * total_days
    fail_ratio = num_violations / total_days

    L0 = (1 - alpha) ** (total_days - num_violations) * alpha ** num_violations
    L1 = (1 - fail_ratio) ** (total_days - num_violations) * fail_ratio ** num_violations

    LR_uc = -2 * (np.log(L0) - np.log(L1))
    p_value = 1 - chi2.cdf(LR_uc, df=1)

print(f"Kupiec LR statistic: {LR_uc:.4f}, p-value: {p_value:.4f}")
if p_value > 0.05:
    print("✅ VaR model passes backtest (failures in line with expectations)")
else:
    print("❌ VaR model fails backtest (too many/few violations)")


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(14, 5))
plt.plot(data.index[-len(rolling_var):], data['TSLA_Return'].iloc[-len(rolling_var):], label="Actual Return")
plt.plot(data.index[-len(rolling_var):], -rolling_var, color='red', label="1-day VaR (95%)")
plt.fill_between(data.index[-len(rolling_var):], -rolling_var, color='red', alpha=0.1)
plt.title("TSLA Returns vs 1-Day GARCH(1,1) VaR")
plt.xlabel("Date")
plt.ylabel("Log Return")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Get regime predictions
regimes = res.smoothed_marginal_probabilities.idxmax(axis=1)

# Plot with colored bands
import matplotlib.pyplot as plt

plt.figure(figsize=(14, 5))
plt.plot(data.index, data['TSLA_Return'], label='TSLA Returns', color='black')

for regime in regimes.unique():
    mask = regimes == regime
    plt.fill_between(data.index, -0.15, 0.15, where=mask, alpha=0.2,
                     label=f"Regime {regime}")

plt.title("TSLA Returns with Regime Classifications")
plt.xlabel("Date")
plt.ylabel("Log Return")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Mark earnings date
plt.axvline(pd.to_datetime("2025-02-01"), color='blue', linestyle='--', label='TSLA Earnings')


In [ ]:
# Regime-based VaR bands
var_r0 = -norm.ppf(0.95) * std_0
var_r1 = -norm.ppf(0.95) * std_1
regime_labels = regimes[-len(data):].values

plt.figure(figsize=(14, 5))
plt.plot(data.index, data['TSLA_Return'], label='Returns', color='black')
plt.plot(data.index, [var_r0 if r == 0 else np.nan for r in regime_labels], label='Regime 0 VaR', color='green')
plt.plot(data.index, [var_r1 if r == 1 else np.nan for r in regime_labels], label='Regime 1 VaR', color='red')
plt.legend()
plt.title("Regime-Based 1-Day VaR")
plt.grid(True)
plt.show()


In [ ]:
plt.figure(figsize=(6, 4))
plt.bar(['GARCH(1,1)', 'Regime Switching'], [-var_garch_95, -var_ms_95], color=['blue', 'green'])
plt.title('1-Day 95% VaR Comparison')
plt.ylabel('Value-at-Risk')
plt.grid(axis='y')
plt.show()


In [ ]:
# Known event dates (you can modify this list)
known_event_dates = pd.to_datetime(["2025-01-08", "2025-02-14", "2025-03-01"])

# Identify regime switches to 1
regime_labels = res.smoothed_marginal_probabilities.idxmax(axis=1)
panic_periods = regime_labels[regime_labels == 1]
panic_groups = panic_periods.groupby((panic_periods != panic_periods.shift()).cumsum())
panic_windows = [(g.index[0], g.index[-1]) for _, g in panic_groups]

# Find panic periods with no known events
panic_no_news = []
for start, end in panic_windows:
    if not any((known_event_dates >= start) & (known_event_dates <= end)):
        panic_no_news.append((start, end))

# Display timeline
for start, end in panic_no_news:
    print(f"Panic zone without event: {start.date()} to {end.date()}")


In [ ]:
durations = regimes.groupby((regimes != regimes.shift()).cumsum()).size()
print(durations.value_counts())


In [ ]:
import matplotlib.pyplot as plt

# Step 1: GARCH conditional volatility
garch_vol = garch_res.conditional_volatility / 100  # Scale back from *100

# Step 2: Regime probabilities
p0 = res.smoothed_marginal_probabilities[0]  # Probability of Regime 0
p1 = res.smoothed_marginal_probabilities[1]  # Probability of Regime 1

# Estimated volatility = weighted average of stds from both regimes
regime_vol = p0 * std_0 + p1 * std_1

# Align indexes (just in case)
garch_vol = garch_vol[-len(regime_vol):]

# Step 3: Plot
plt.figure(figsize=(14, 5))
plt.plot(regime_vol.index, regime_vol, label='Regime-Switching Volatility', color='green')
plt.plot(garch_vol.index, garch_vol, label='GARCH(1,1) Volatility', color='blue', alpha=0.7)

plt.title('Volatility Comparison: Regime-Switching vs GARCH')
plt.xlabel('Date')
plt.ylabel('Volatility (Std Dev)')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
durations.plot(kind='hist', bins=20, title='Distribution of Regime Durations')


In [ ]:
# Label most probable regime (0 = low-vol, 1 = high-vol)
regimes = res.smoothed_marginal_probabilities.idxmax(axis=1)


In [ ]:
# Identify continuous Regime 1 periods (panic regimes)
panic_periods = regimes[regimes == 1]
panic_groups = panic_periods.groupby((panic_periods != panic_periods.shift()).cumsum())

# Get start-end dates of each panic window
panic_windows = [(group.index[0], group.index[-1]) for _, group in panic_groups]
print("Detected high-volatility regimes (potential panic):")
for start, end in panic_windows:
    print(f"{start.date()} to {end.date()}")


In [ ]:
# Simulate known headline dates (pretend Elon tweeted or earnings came out)
known_event_dates = [
    "2025-01-08", "2025-02-14", "2025-03-01"  # Customize as needed
]
known_event_dates = pd.to_datetime(known_event_dates)

# Find panic windows without news
panic_no_news = []
for start, end in panic_windows:
    if not any((known_event_dates >= start) & (known_event_dates <= end)):
        panic_no_news.append((start, end))

print("\n🧠 Panic regimes WITHOUT any known event:")
for start, end in panic_no_news:
    print(f"{start.date()} to {end.date()}")


In [ ]:
# Transition probabilities (already computed)
from scipy.special import expit
p00 = expit(res.params[-2])  # Stay in Regime 0
p11 = expit(res.params[-1])  # Stay in Regime 1

# Last known regime (most likely)
last_prob = res.smoothed_marginal_probabilities.iloc[-1]
last_regime = last_prob.idxmax()

# Predict 5-day forward regime probability
pred = np.zeros(2)
pred[last_regime] = 1

for _ in range(5):
    pred = np.array([pred[0]*p00 + pred[1]*(1 - p11),
                     pred[0]*(1 - p00) + pred[1]*p11])

print(f"📊 Probability TSLA stays in Regime 0 (low volatility) over next 5 days: {pred[0]:.2%}")


In [ ]:
regimes = res.smoothed_marginal_probabilities.idxmax(axis=1)
data['Regime'] = regimes

mean_returns_by_regime = data.groupby('Regime')['TSLA_Return'].mean()
print(mean_returns_by_regime)


In [ ]:
# Model comparison between GARCH(1,1) and Markov-switching model

ll_garch = garch_res.loglikelihood
aic_garch = garch_res.aic
bic_garch = garch_res.bic

ll_markov = res.llf
aic_markov = res.aic
bic_markov = res.bic

print("GARCH(1,1) model:")
print(f"  log-likelihood = {ll_garch:.2f}")
print(f"  AIC            = {aic_garch:.2f}")
print(f"  BIC            = {bic_garch:.2f}")

print("\nMarkov-switching model:")
print(f"  log-likelihood = {ll_markov:.2f}")
print(f"  AIC            = {aic_markov:.2f}")
print(f"  BIC            = {bic_markov:.2f}")


In [ ]:
# Expected regime durations from transition probabilities

exp_dur_0 = 1.0 / (1.0 - p00)
exp_dur_1 = 1.0 / (1.0 - p11)

print(f"Expected duration in Regime 0 (low volatility):  {exp_dur_0:.1f} trading days")
print(f"Expected duration in Regime 1 (high volatility): {exp_dur_1:.1f} trading days")
